### Preliminary functions

In [12]:
# this is a comment
#this is a test

t = var('t')
R_jones = Matrix(SR, 4, 4, [t^(1/2),0,0,0, 0,0,t,0, 0,t,t^(1/2)-t^(3/2),0, 
0,0,0,t^(1/2)])
R_alex = Matrix(SR, 4, 4, [t^(-1/2),0,0,0, 0,0,1,0, 0,1,t^(-1/2)-t^(1/2),0, 
0,0,0,-t^(1/2)])
h_jones = Matrix(SR, 2, 2, [t^(-1/2),0, 0,t^(1/2)])
h_alex = Matrix(SR, 2, 2, [t^(1/2),0, 0,-t^(1/2)])

def t_prod(A, B):
    '''
    Parameters
    ----------
    A : Matrix
        first matrix.
    B : Matrix
        second matrix

    Returns
    -------
    AtB_final : Matrix
        kronecker product of A and B
    '''
    m=A.nrows()
    n=A.ncols()
    p=B.nrows()
    q=B.ncols()
    AtB_list = []
    
    for i in range(m):
        ith_blocks = []
        
        for j in range(n):
            AijB=A[i,j]*B
            ith_blocks.append(AijB)
            
        for k in range(p): 
            for block in ith_blocks:
                AtB_list.append(block[k])
    
    flat = [item for sublist in AtB_list for item in sublist]
    AtB_final = Matrix(SR, p*m, q*n, flat)
    
    return AtB_final

def nth_tens(A, n):
    '''
    Parameters
    ----------
    A : Matrix
        matrix to tensor.
    n : int
        power of tensor product

    Returns
    -------
    newA : Matrix
        A tensored with itself n times
    '''
    newA = A
    if n == 0:
        return Matrix(SR, 1, 1, [1])
    for i in range(n-1):
        newA = t_prod(newA,A)
    return newA

def prods(l):
    '''
    Parameters
    ----------
    l : list
        list of matrices to tensor.

    Returns
    -------
    Matrix
        the tensor product of the matrices in l
    '''
    while len(l) > 1:
        l_new = l[:-2]
        last = t_prod(l[-2], l[-1])
        l_new.append(last)
        l = l_new
        
    return l[0]

def nth_permutation(n):
    '''
    Parameters
    ----------
    n : int
        dimension of V
        
    Returns
    -------
    P : Matrix
        permutation matrix in End(VxV)
        e_i x e_j --> e_j x e_i 
    '''
    P = Matrix(SR, n^2, n^2)
    while P[n^2 - 1, n^2 - 1] != 1:
        for i in range(n):
            P[i*n, i] = 1
            for j in range(n):
                P[i*n + j, i + j*n] = 1
        
    return P

### Jones/Alexander polynomials from braid reps

In [13]:
def braid(el, n, unity):
    '''
    Parameters
    ----------
    el : list
        list of generators forming the braid rep
        e.g. (s_1) (s_2)^(-1) (s_3)^2 (s_2) --> [1, -2, 3, 3, 2]
    n : int
        number of strands (el in B_n)
    unity : bool
        True if Alexander polynomial, False if Jones polynomial

    Returns
    -------
    poly : expr
        Alexander polynomial if unity
        ((t^(1/2)+t^(-1/2)) * Jones polynomial) otherwise
    '''
    prod = []
    idd = identity_matrix(2)
    for i in el:
        inv = False
        if i < 0:
            inv = True
            i = abs(i)
        l = nth_tens(idd, i-1)
        r = nth_tens(idd, n-i-1)
        
        if inv:
            if unity:
                phi = t_prod(l, t_prod(R_alex.inverse(), r))
            else:
                phi = t_prod(l, t_prod(R_jones.inverse(), r))
        else:
            if unity:
                phi = t_prod(l, t_prod(R_alex, r))
            else:
                phi = t_prod(l, t_prod(R_jones, r))
        prod.append(phi)
    
    phib = identity_matrix(2**n)
    for p in prod:
        phib = phib*p
    
    if unity:
        hphi = t_prod(idd, nth_tens(h_alex, n-1))*phib
        poly = almost_trace(hphi)
        if not bool(poly[0,0] == poly[1,1]):
            print("whoops!") # checks the matrix is a multiple of id_V
            return None
        else:
            poly = poly[0,0]
    else:
        hphi = nth_tens(h_jones, n)*phib
        poly = hphi.trace()
    
    return (poly.expand()).simplify()

def almost_trace(A):
    '''
    Parameters
    ----------
    A : Matrix
        2^n x 2^n matrix (in End(VxVx...xV)) to 'almost trace' over 
        (see Returns)
        
    Returns
    -------
    M : matrix
        trace_{2, 3, ..., n}(A)
    '''
    n=A.nrows()
    m = int(n/2)
    M = Matrix(SR, 2, 2)
    for i in range(2):
        for j in range(2):
            M[i,j] = sum([A[k+i*m,k+j*m] for k in range(m)])
    return M

factor = t^(1/2)+t^(-1/2)

# 3_1
#pretty_print(braid([1, 1, 1], 2, False))
#pretty_print((factor*(t+t^3-t^4)).expand())
#pretty_print(braid([1, 1, 1], 2, True)) 
#pretty_print((t-1+(t^-1)).expand())

# 4_1
#pretty_print(braid([1, -2, 1, -2], 3, False))
#pretty_print((factor*(t^2-t+1-t^(-1)+t^(-2))).expand())
#pretty_print(braid([1, -2, 1, -2], 3, True))
#pretty_print((-t^(-1)+3-t).expand())

# 5_1
#pretty_print(braid([-1, -1, -1, -1, -1], 2, False))
#pretty_print((factor*(-t^(-7)+t^(-6)-t^(-5)+t^(-4)+t^(-2))).expand())
#pretty_print(braid([-1, -1, -1, -1, -1], 2, True))
#pretty_print((t^2+t^(-2)-t-t^(-1)+1).expand())

# 5_2 
#pretty_print(braid([-1, -1, -2, -2, -1, 2], 3, False))
#x = (factor*(-t^(-6)+t^(-5)-t^(-4)+2*(t^(-3))-t^(-2)+t^(-1))).expand()
#pretty_print(x)
#pretty_print(braid([-1, -1, -2, -2, -1, 2], 3, True))
#pretty_print((2*t+2*t^(-1)-3).expand())

### Jones/Alexander polynomials from tangle diagrams

In [14]:
# Jones
n = Matrix(SR, 1, 4, [t^(-1/2),0,0,t^(1/2)])
n_ = Matrix(SR, 1, 4, [1,0,0,1])
u = Matrix(SR, 4, 1, [t^(1/2),0,0,t^(-1/2)])
u_ = Matrix(SR, 4, 1, [1,0,0,1])

idd = identity_matrix(2)

# 3_1 jones - cups and caps 
a1 = t_prod(n_, n)
a2 = t_prod(idd, t_prod(R_jones, idd))
a3 = t_prod(u, u_)

f = a1*a2*a2*a2*a3
#pretty_print(f.expand()) 

# 4_1 jones - cups and caps
a1 = t_prod(n_, n)
a2 = prods([idd, R_jones, idd])
a3 = prods([idd, idd, idd, n, idd])
a4 = prods([idd, idd, R_jones.inverse(), idd, idd])
a5 = prods([idd, R_jones, idd, idd, idd])
a6 = prods([idd, idd, idd, u_, idd])
a7 = t_prod(u, u_)

f = a1*a2*a3*a4*a5*a4*a6*a7
#pretty_print(f.expand()) 

# 5_1 jones
a1 = t_prod(n_, n)
a2 = prods([idd, R_jones.inverse(), idd])
a3 = t_prod(u, u_)

f = a1*(a2^5)*a3
#pretty_print(f.expand())

# 5_2 jones
a1 = t_prod(n_, n)
a2 = prods([idd, idd, idd, n, idd])
a3 = prods([idd, R_jones.inverse(), idd, idd, idd])
a4 = prods([idd, idd, R_jones.inverse(), idd, idd])
a5 = prods([idd, idd, R_jones, idd, idd])
a6 = prods([idd, idd, idd, u_, idd])
a7 = t_prod(u, u_)

f = a1*a2*(a3^2)*(a4^2)*a3*a5*a6*a7
#pretty_print(f.expand())

# ALexander
n = Matrix(SR, 1, 4, [t^(1/2),0,0,-t^(1/2)])
n_ = Matrix(SR, 1, 4, [1,0,0,1])
u = Matrix(SR, 4, 1, [-t^(1/2),0,0,t^(1/2)])
u_ = Matrix(SR, 4, 1, [1,0,0,1])

# 3_1 alexander - 1-tangle 
a1 = t_prod(idd, n)
a2 = t_prod(R_alex.inverse(), idd)
a3 = t_prod(idd, u_)

f = a1*a2*a2*a2*a3
#pretty_print(f[0, 0].expand())

# 4_1 alexander - 1-tangle
a1 = t_prod(idd, n)
a2 = t_prod(R_alex, idd)
a3 = prods([idd, idd, n, idd])
a4 = prods([idd, R_alex.inverse(), idd, idd])
a5 = prods([R_alex, idd, idd, idd])
a6 = prods([idd, idd, u_, idd])
a7 = t_prod(idd, u_)

f = a1*a2*a3*a4*a5*a4*a6*a7
#pretty_print(f[0,0].expand())

# 5_1 alexander - 1-tangle
a1 = t_prod(idd, n)
a2 = t_prod(R_alex.inverse(), idd)
a3 = t_prod(idd, u_)

f = a1*(a2^5)*a3
#pretty_print(f[0,0].expand())

# 5_2 alexander 
a1 = t_prod(idd, n)
a2 = prods([idd, idd, n, idd])
a3 = prods([R_alex.inverse(), idd, idd, idd])
a4 = prods([idd, R_alex.inverse(), idd, idd])
a5 = prods([idd, R_alex, idd, idd])
a6 = prods([idd, idd, u_, idd])
a7 = t_prod(idd, u_)

f = a1*a2*(a3^2)*(a4^2)*a3*a5*a6*a7
#pretty_print(f[0,0].expand())

### Jones/Alexander polynomials from $U_q(sl_2)$ 

In [15]:
q = var('q')
xi = var('x') 
lam = var('l')
# sage won't simplify expressions if given the variables as unicodes
# so i redefine these at the end

def quantum(qq, n):
    '''
    Parameters
    ----------
    qq : var
        q or xi (depending on unity).
    n : int
        integer to quantise.

    Returns
    -------
    bracket : expr
        [n]
    '''
    bracket = (qq^(n/2) - qq^(-n/2))/(qq^(1/2)-qq^(-1/2))
    return bracket

def quantum_fact(qq, n):
    '''
    Parameters
    ----------
    qq : var
        q or xi (depending on unity).
    n : int
        integer to quantum factorial.

    Returns
    -------
    p : expr
        [n]!.
    '''
    p = 1
    for i in range(1, n+1):
        p*= quantum(qq, i)
    return p

def exp_qq(qq, x, n):
    '''
    Parameters
    ----------
    qq : var
        q or xi (depending on unity).
    x : expr
        expression to exponentiate.
    n : int
        upper limit (dimension or r : xi^r = 1).

    Returns
    -------
    s : expr
        exp_qq (x).
    '''
    s = 0
    for i in range(n):
        s+= ((qq^(i*(i-1)/4))/(quantum_fact(qq, i)))*(x^i)
    return s

def p_Vn_E(n, unity):
    '''
    Parameters
    ----------
    n : int
        dimension or r (depending on unity).
    unity : bool
        True if xi is a root of unity (alexander), false otherwise (jones).

    Returns
    -------
    Matrix
        n-dimensional rho(E).
    '''
    E = Matrix(SR, n, n)
    for i in range(n):
        if i < n-1:
            if unity:
                E[i,i+1] = quantum(xi, n-i-1+lam)
            else:
                E[i,i+1] = quantum(q, n - i-1)
                
    if unity:
        return E
    else:  
        return E.simplify_full()

def p_Vn_F(n, unity):
    '''
    Parameters
    ----------
    n : int
        dimension or r (depending on unity).
    unity : bool
        True if xi is a root of unity (alexander), false otherwise (jones).

    Returns
    -------
    Matrix
        n-dimensional rho(F).
    '''
    F = Matrix(SR, n, n)
    for i in range(n):
        if i > 0:
            if unity:
                F[i,i-1] = quantum(xi, i)
            else:
                F[i,i-1] = quantum(q, i)
    return F.simplify_full()

def p_Vn_K(n, unity):
    '''
    Parameters
    ----------
    n : int
        dimension or r (depending on unity).
    unity : bool
        True if xi is a root of unity (alexander), false otherwise (jones).

    Returns
    -------
    Matrix
        n-dimensional rho(K).
    '''
    K = Matrix(SR, n, n)
    for i in range(n):
        if unity:
            K[i,i] = xi^((n-(2*i+1)+lam)/2)
        else:
            K[i,i] = q^((n-(2*i+1))/2)
    
    if unity:
        return K
    else:
        return K.simplify_full()

def p_Vn_H(n, unity):
    '''
    Parameters
    ----------
    n : int
        dimension or r (depending on unity).
    unity : bool
        True if xi is a root of unity (alexander), false otherwise (jones).

    Returns
    -------
    Matrix
        n-dimensional rho(H).
    '''
    H = Matrix(SR, n, n)
    for i in range(n):
        if unity: 
            H[i,i] = n-(2*i+1)+lam
        else:
            H[i,i] = n-(2*i+1)
    return H

def qq_pow_mat(qq, diag):
    '''
    Parameters
    ----------
    qq : var
        q or xi (depending on unity).
    diag : Matrix
        diagonal matrix to exponentiate.

    Returns
    -------
    diag : Matrix
        qq^diag.
    '''
    for i in range(diag.nrows()):
        diag[i,i] = qq^(diag[i,i])
        
    return diag

def R(n, unity):
    '''
    Parameters
    ----------
    n : int
        dimension or r (depending on unity).
    unity : bool
        True if xi is a root of unity (alexander), false otherwise (jones).

    Returns
    -------
    Matrix
        n^2-dimensional (rho x rho)(R).
    '''
    # variables below are so that the equation doesn't run over the page
    po = (1/4)*t_prod(p_Vn_H(n, unity), p_Vn_H(n, unity))
    af = t_prod(p_Vn_E(n, unity), p_Vn_F(n, unity))
    if unity:
        R = qq_pow_mat(xi,po)*exp_qq(xi,(xi^(1/2)-xi^(-1/2))*af,n)
        return R
    else:
        R = qq_pow_mat(q,po)*exp_qq(q,(q^(1/2)-q^(-1/2))*af,n)
        return R.simplify_full()

def u(n, unity):
    '''
    Parameters
    ----------
    n : int
        dimension or r (depending on unity).
    unity : bool
        True if xi is a root of unity (alexander), false otherwise (jones).

    Returns
    -------
    Matrix
        n-dimensional rho(u).
    '''
    s = 0
    for i in range(n):
        if unity:
            fr = ((xi^(-1/2)-xi^(1/2))^i)/quantum_fact(xi,i)
            af = (p_Vn_K(n, unity)^(-i))*(p_Vn_E(n,unity)^i)
            s += xi^(3*i*(i-1)/4)*fr*(p_Vn_F(n,unity)^i)*af
        else:
            fr = ((q^(-1/2)-q^(1/2))^i)/quantum_fact(q, i)
            af = (p_Vn_K(n, unity)^(-i))*(p_Vn_E(n, unity)^i)
            s += q^(3*i*(i-1)/4)*fr*(p_Vn_F(n, unity)^i)*af
    
    if unity:
        uu = qq_pow_mat(xi, (-1/4)*(p_Vn_H(n, unity)^2))*s
        return uu.simplify_full()
    else:
        uu = qq_pow_mat(q, (-1/4)*(p_Vn_H(n, unity)^2))*s
        return uu.simplify_full()

def v(n, unity):
    '''
    Parameters
    ----------
    n : int
        dimension or r (depending on unity).
    unity : bool
        True if xi is a root of unity (alexander), false otherwise (jones).

    Returns
    -------
    Matrix
        n-dimensional rho(v).
    '''
    if unity:
        vv = (p_Vn_K(n, unity)^(n-1))*u(n, unity)
    else:
        vv = (p_Vn_K(n, unity)^(-1))*u(n, unity)
    return vv

P = Matrix(SR, 4, 4, [1,0,0,0, 0,0,1,0, 0,1,0,0, 0,0,0,1])
R_jones = P*R(2, False)
h_jones = p_Vn_K(2, False)
#pretty_print(R_jones)
#pretty_print(h_jones)

l = var('\u03BB')
x = var('\u03B6') # right letter now...

R_alex = (P*R(2, True)).subs(xi==x, lam==l)
h_alex = (p_Vn_K(2, True)^(1-2)).subs(xi==x, lam==l) #1-r
#pretty_print(R_alex)
#pretty_print(h_alex)

normal_jones = v(2, False)[0,0]
normal_alex = (v(2, True)[0,0]).simplify_full().subs(xi==x, lam==l)
#pretty_print(normal_jones)
#pretty_print(normal_alex)

In [16]:
# playing around with 3d V

P = nth_permutation(3)
R_jones = P*R(3, False)
h_jones = p_Vn_K(3, False)
#pretty_print(R_jones)
#pretty_print(h_jones)
R_alex = P*R(3, True)
h_alex = (p_Vn_K(3, True))^(-2)
#pretty_print(R_alex.subs(xi==x, lam==l))
#pretty_print(h_alex.subs(xi==x, lam==1))

a = var('a')
b = var('b')

#pretty_print((a*R_jones - b*(R_jones.inverse())).simplify_full())
#pretty_print((a*R_alex - (a*(xi^((1/2)*(lam^2)+lam))*(R_alex.inverse()))).simplify_full().subs(xi==x, lam==l))
# doesn't seem to satify a skein relation :( (neither 3d "jones" nor "alexander")

def finds_basis(n, m):
    start = [0]*m
    basis = [start]
    while len(basis) < n^m:
        for i in range(n):
            count = 0
            t = basis[-1].copy()
            for s in t:
                if s < n-1:
                    t[count] += 1
                    basis.append(t)
                    #print(basis)
                    break
                else:
                    for j in range(count+1):
                        t[j] = 0
                count += 1
    
    ordered = [list(reversed(k)) for k in basis]
    return ordered

def charge_conserved_subspaces(n, m):
    '''
    Parameters
    ----------
    n : int
        dimension of V
    m : int
        number of times V is tensored with itself
        
    Returns
    -------
    subspaces : list
        list of subspaces preserved by a charge-conserving linear map
    int
        len(subspaces)
    '''
    ordered = finds_basis(n, m)
    subspaces = []
    
    for k in range(((n-1)*m)+1):
        subspace = []
        for l in ordered:
            if sum(l) == k:
                subspace.append(l)
        subspaces.append(subspace)
        
    return subspaces, len(subspaces)

#print(charge_conserved_subspaces(2, 3))

a, b, c, d, e, f, g, h, i, j, k, l, m, n, p, q, r, s, t = var('a b c d e f g h i j k l m n p q r s t')
#R x id
M1 = Matrix(SR, 1, 1, [a])
M2 = Matrix(SR, 3, 3, [a,0,0, 0,b,c, 0,d,j])
M3 = Matrix(SR, 6, 6, [a,0,0,0,0,0, 0,b,0,c,0,0, 0,0,e,0,f,g, 0,d,0,j,0,0, 0,0,h,0,k,l, 0,0,i,0,m,r])
M4 = Matrix(SR, 7, 7, [b,0,c,0,0,0,0, 0,e,0,f,0,g,0, d,0,j,0,0,0,0, 0,0,0,k,0,l,0, 0,0,0,0,n,0,p, 0,i,0,m,0,r,0, 0,0,0,0,q,0,s])
M5 = Matrix(SR, 6, 6, [e,f,0,g,0,0, h,k,0,l,0,0, 0,0,n,0,p,0, i,m,0,r,0,0, 0,0,q,0,s,0, 0,0,0,0,0,t])
M6 = Matrix(SR, 3, 3, [n,p,0, q,s,0, 0,0,t])
M7 = Matrix(SR, 1, 1, [t])

#id x R
A1 = Matrix(SR, 1, 1, [a])
A2 = Matrix(SR, 3, 3, [b,c,0, d,j,0, 0,0,a])
A3 = Matrix(SR, 6, 6, [e,f,g,0,0,0, h,k,l,0,0,0, i,m,r,0,0,0, 0,0,0,b,c,0, 0,0,0,d,j,0, 0,0,0,0,0,a])
A4 = Matrix(SR, 7, 7, [n,p,0,0,0,0,0, q,s,0,0,0,0,0, 0,0,e,f,g,0,0, 0,0,h,k,l,0,0, 0,0,i,m,r,0,0, 0,0,0,0,0,b,c, 0,0,0,0,0,d,j])
A5 = Matrix(SR, 6, 6, [t,0,0,0,0,0, 0,n,p,0,0,0, 0,q,s,0,0,0, 0,0,0,e,f,g, 0,0,0,h,k,l, 0,0,0,i,m,r])
A6 = Matrix(SR, 3, 3, [t,0,0, 0,n,p, 0,q,s])
A7 = Matrix(SR, 1, 1, [t])

ML = [M1, M2, M3, M4, M5, M6, M7]
AL = [A1, A2, A3, A4, A5, A6, A7]

#for i in range(7):
#    pretty_print(ML[i]*AL[i]*ML[i])
#    pretty_print(AL[i]*ML[i]*AL[i])
#    print("\n")

#pretty_print(solve(M2*A2*M2 = A2*M2*A2, b))

### Skein relations

In [17]:
# more playing around
# generalise below?
def skein_checker(n):
    '''
    Parameters
    ----------
    n : int
        dimension of V
  
    Returns
    -------
    dimension : int
        dimension of the subspace of elements which commute with the 
        action of U_q(sl_2)
    '''
    I = identity_matrix(n)
    E = p_Vn_E(n, False)
    F = p_Vn_F(n, False)
    K = p_Vn_K(n, False)
    
    s = ""
    for i in range(1, n^4 + 1):
        s += " a" + str(i)
    s = s[1:]
    vv = var(s)
    general_mat = Matrix(SR, n^2, n^2, list(vv))
    eqns = []
    delts = [t_prod(I,I),t_prod(E,K)+t_prod(I,E)]
    delts.append(t_prod(F,I)+t_prod(K^(-1),F))
    delts.append(t_prod(K,K))
    #pretty_print(delts)
    for d in delts:
        dF = d*general_mat
        Fd = general_mat*d
        for i in range(n^2):
            for j in range(n^2):
                eqns.append(dF[i,j]==Fd[i,j])
    
    sols = solve(eqns,vv)[0]
    vs = [i.variables() for i in sols]
    flat_vs = [i for j in vs for i in j]
    ar_set = set(flat_vs)
    r_set = set([el for el in ar_set if str(el)[0] == "r"])
    
    #return(solve(eqns, vv)) # uncomment to see the form of the solutions
    dimension = len(r_set)
    return dimension

#pretty_print(skein_checker(2))
#pretty_print(skein_checker(3))

### Hopf superalgebras - preliminary functions

In [18]:
def supertrace(M, parity_basis_V):
    '''
    Parameters
    ----------
    M : matrix
        matrix to supertrace over
    parity_basis_V : list
        the parity of each element of the basis of V 
        (e.g. [0,1] for V = C<1,psi>)
  
    Returns
    -------
    tr : expr
        the supertrace of M
    '''
    m = len(parity_basis_V)
    n = M.nrows()
    tr = 0
    for i in range(n):
        if m^i == n:
            break
    parity_basis = parity_basis_Vn(parity_basis_V, i)
    for j in range(n):
        tr += ((-1)^(parity_basis[j]))*M[j,j]
    return tr

def almost_supertrace(M, parity_basis_V):
    '''
    Parameters
    ----------
    M : matrix
        matrix to "almost supertrace" over
    parity_basis_V : list
        the parity of each element of the basis of V 
        (e.g. [0,1] for V = C<1,psi>)
  
    Returns
    -------
    Str2 : expr
        str_{2,3,...,n}(M)
    '''
    m = len(parity_basis_V)
    n = 0
    while m^n < M.nrows():
        n+=1 
    parity_basis2 = parity_basis_Vn(parity_basis_V, n-1)
    Str2 = Matrix(SR, m, m)
    for i in range(m):
        for j in range(m):
            s = 0
            for k in range(m^(n-1)):
                Mi = k + i*int(M.nrows()/m)
                Mj = k + j*int(M.nrows()/m)
                parity = parity_basis2[k]
                s += ((-1)^parity)*M[Mi,Mj]
            Str2[i,j] = s
    
    return Str2

def parity_basis_Vn(parity_basis_V, n):
    '''
    Parameters
    ----------
    parity_basis_V : list
        the parity of each element of the basis of V 
        (e.g. [0,1] for V = C<1,psi>)
    n : int
        number of times V has been tensored with itself
  
    Returns
    -------
    pb : list
        a list of the parity of each element of the basis of V^{(x)n}
    '''
    m = len(parity_basis_V)
    parity_basis = parity_basis_V
    while len(parity_basis) < m^n:
        new_parity_basis = []
        for el1 in parity_basis:
            for el2 in parity_basis_V:
                new_parity_basis.append(el1+el2)
        parity_basis = new_parity_basis
    
    pb = [i % 2 for i in parity_basis]

    return pb

def super_nth_permutation(parity_basis_V):
    '''
    Parameters
    ----------
    parity_basis_V : list
        the parity of each element of the basis of V 
        (e.g. [0,1] for V = C<1,psi>)
  
    Returns
    -------
    P : Matrix
        super permutation matrix in End(VxV)
        e_i x e_j --> (-1)^(|e_j||e_i|)e_j x e_i 
    '''
    n = len(parity_basis_V)
    P = Matrix(SR, n^2, n^2)
    pb = []
    for i in parity_basis_V:
        for j in parity_basis_V:
            pb.append(i+j)
    npb = []
    for p in pb:
        if p == 2:
            npb.append(1)
        else:
            npb.append(0)
    while P[n^2 - 1, n^2 - 1] == 0:
        for i in range(n):
            P[i*n, i] = (-1)^(npb[i*n]*npb[i])
            for j in range(n):
                P[i*n + j, i + j*n] = (-1)^(npb[i*n + j]*npb[i + j*n])
        
    return P

### Hopf superalgebras

In [19]:
def psi_pos(pos, P):
    '''
    Parameters
    ----------
    pos : bool
        True if W+ rep, False is W- rep (doesn't matter if P rep)
    P : bool
        True if P rep, False otherwise
  
    Returns
    -------
    psi : Matrix
        rep of psi+
    '''
    if P:
        psi = Matrix(SR, 4, 4, [0,0,0,0, 1,0,0,0, 0,0,0,0, 0,0,1,0])
        return psi
    else:
        if pos:
            psi = Matrix(SR, 2, 2, [0,0, 1,0])
        else:
            psi = zero_matrix(2)
        
        return psi
            
def psi_neg(pos, P):
    '''
    Parameters
    ----------
    pos : bool
        True if W+ rep, False is W- rep (doesn't matter if P rep)
    P : bool
        True if P rep, False otherwise
  
    Returns
    -------
    psi : Matrix
        rep of psi-
    '''
    if P:
        psi = Matrix(SR, 4, 4, [0,0,0,0, 0,0,0,0, 1,0,0,0, 0,-1,0,0])
        return psi
    else:
        return psi_pos(not(pos), False)
    
def D(Mat):
    '''
    Parameters
    ----------
    Mat : matrix
        matrix to find the coproduct of
  
    Returns
    -------
    M : Matrix
        Delta(Mat)
    '''
    I = identity_matrix(Mat.nrows())
    M = t_prod(Mat, I) + t_prod(I, Mat)
    return M

def S(Mat):
    '''
    Parameters
    ----------
    Mat : matrix
        matrix to find the antipode of
  
    Returns
    -------
    Matrix
        S(Mat)
    '''
    return (-1)*Mat

def Rm(pos, P):
    '''
    Parameters
    ----------
    pos : bool
        True if W+ rep, False is W- rep (doesn't matter if P rep)
    P : bool
        True if P rep, False otherwise
  
    Returns
    -------
    Rmat : Matrix
        rep of R
    '''
    psi_p = psi_pos(pos, P)
    psi_n = psi_neg(pos, P)
    d = (psi_p.nrows())^2
    if P:
        parity_basis_V = [0,1,1,0] # V = <1,p,n,pn>
    else:
        parity_basis_V = [0,1] # V = <1,psi>
    sP = super_nth_permutation(parity_basis_V)
    Rmat = sP*(identity_matrix(d) - t_prod(psi_p, psi_n))
    return Rmat

#pretty_print(Rm(True, False)) # W+ (x) W+
#pretty_print(Rm(False, False)) # W- (x) W-
iddd = identity_matrix(4)
ip = iddd - t_prod(psi_pos(True, False), psi_neg(False, False))
RR = super_nth_permutation([0,1])*ip
#pretty_print(RR) # W+ (x) W-
ip = iddd - t_prod(psi_pos(False, False), psi_neg(True, False))
RR = super_nth_permutation([0,1])*ip
#pretty_print(RR) # W- (x) W+
#pretty_print(Rm(True, True)) # P

def super_skein_checker(pos, P):
    '''
    Parameters
    ----------
    pos : bool
        True if W+ rep, False is W- rep (doesn't matter if P rep)
    P : bool
        True if P rep, False otherwise
  
    Returns
    -------
    int
        dimension of subspace which commutes w action of psl(1|1)
    '''
    psi_p = psi_pos(pos, P)
    psi_n = psi_neg(pos, P)
    n = psi_n.nrows()
    I = identity_matrix(n)
    els = [psi_p, psi_n, I]
    
    s = ""
    for i in range(1, n^4 + 1):
        s += " a" + str(i)
    s = s[1:]
    vv = var(s)
    general_mat = Matrix(SR, n^2, n^2, list(vv))
    eqns = []
    delts = [D(el) for el in els]
    for d in delts:
        dF = d*general_mat
        Fd = general_mat*d
        for i in range(n^2):
            for j in range(n^2):
                eqns.append(dF[i,j]==Fd[i,j])
    
    sols = solve(eqns,vv)[0]
    vs = [i.variables() for i in sols]
    flat_vs = [i for j in vs for i in j]
    ar_set = set(flat_vs)
    r_set = set([el for el in ar_set if str(el)[0] == "r"])
    
    #return(solve(eqns, vv))
    return len(r_set)

# below: dimensions of subspaces | D(a).F = F.D(a)
#print(super_skein_checker(True, False)) # W+
#print(super_skein_checker(False, False)) # W-
#print(super_skein_checker(True, True)) # P

def superbraid(el, n, Rmat, hmat, unity, parity_basis_V):
    '''
    Parameters
    ----------
    el : list
        list of generators forming the braid rep
        e.g. (s_1) (s_2)^(-1) (s_3)^2 (s_2) --> [1, -2, 3, 3, 2]
    n : int
        number of strands (el in B_n)
    Rmat : Matrix
        R matrix
    hmat : Matrix
        h matrix
    unity : bool
        True to supertrace, False to just trace
        (supertrace should always be zero, so set this as True!)
        (False option was just for testing)
    party_basis_V : list
        the parity of each element of the basis of V 
        (e.g. [0,1] for V = C<1,psi>)

    Returns
    -------
    poly : expr
        invariant from R and h
    '''
    prod = []
    d = hmat.nrows()
    idd = identity_matrix(d)
    c = 0
    for i in el:
        c += 1 
        inv = False
        if i < 0:
            inv = True
            i = abs(i)
        l = nth_tens(idd, i-1)
        r = nth_tens(idd, n-i-1)
        
        if inv:
             phi = t_prod(l, t_prod(Rmat.inverse(), r))

        else:
            phi = t_prod(l, t_prod(Rmat, r))
                
        prod.append(phi)
    
    phib = identity_matrix(d**n)
    for p in prod:
        phib = phib*p
    
    if unity:
        hphi = t_prod(idd, nth_tens(hmat, n-1))*phib
        poly = almost_supertrace(hphi,parity_basis_V)
        if not bool(poly[0,0] == poly[1,1]):
            print("whoops!") # checks the matrix is a multiple of id_V
            return None
        else:
            poly = poly[0,0]
    else:
        hphi = nth_tens(hmat, n)*phib
        poly = supertrace(hphi, parity_basis_V)
    
    return (poly.expand()).simplify()

idd = identity_matrix(2)
iddd = identity_matrix(4)
b = [1, 1, 1] # 3_1
#pretty_print(superbraid(b, 2, Rm(True, True), iddd, True, [0,1,1,0])) 
b = [1, -2, 1, -2] # 4_1
#pretty_print(superbraid(b, 3, Rm(True, True), iddd, True, [0,1,1,0])) 
b = [-1, -1, -1, -1, -1] # 5_1
#pretty_print(superbraid(b, 2, Rm(True, True), iddd, True, [0,1,1,0]))
b = [-1, -1, -2, -2, -1, 2] # 5_2
#pretty_print(superbraid(b, 3, Rm(True, True), iddd, True, [0,1,1,0])) 
b = [-3, -3, -2, 1, 3, -2, 1] # 6_1
#pretty_print(superbraid(b, 4, Rm(True, True), iddd, True))
b = [-2, -2, -2, 1, -2, 1] # 6_2
#pretty_print(superbraid(b, 3, Rm(True, True), iddd, True, [0,1,1,0]))
b = [-2, -2, 1, -2, 1, 1] # 6_3
#pretty_print(superbraid(b, 3, Rm(True, True), iddd, True, [0,1,1,0]))
b = [-1, -1, -1, -1, -1, -1, -1] # 7_1
#pretty_print(superbraid(b, 2, Rm(True, True), iddd, True, [0,1,1,0])) 
b = [-3, -3, -3, -2, 3, -2, -1, 2, -1] #7_2
#pretty_print(superbraid(b, 4, Rm(True, True), iddd, False)) #7_2
# up to 7_2 - braids w 4 strands seem to be too much for my computer
# which is why they're commented out here.
# (Sage is having to deal with 256x256 matrices, to be fair)

# links
RR = Rm(True, True)
#pretty_print(RR) # W+ (x) W-
#pretty_print(RR.inverse())
#pretty_print(RR-RR^(-1))
b = [1, 1] # L2a1
pretty_print(superbraid(b, 2, RR, iddd, True, [0,1]))
b = [3, -2, 1, -2, -3, -2, -1, -2] # L4a1
#pretty_print(superbraid(b, 4, RR, iddd, True, [0,1]))
b = [-2, 1, -2, 1, -2] # L5a1
pretty_print(superbraid(b, 3, RR, iddd, True, [0,1]))
b = [3, -2, 1, -2, 3, -2, -1, -2] # L6a1
#pretty_print(superbraid(b, 4, RR, iddd, True, [0,1]))
b = [3, -2, -2, -2, 1, -2, -3, -2, -1, -2] # L6a2
#pretty_print(superbraid(b, 4, RR, iddd, True, [0,1]))
b = [-1, -1, -1, -1, -1, -1] # L6a3
pretty_print(superbraid(b, 2, RR, iddd, True, [0,1]))
b = [2, -1, 2, -1, 2, -1] # L6a4
pretty_print(superbraid(b, 3, RR, iddd, True, [0,1]))
# L6a5
b = [2, -1, 2, 1, -2, 1] # L6n1
pretty_print(superbraid(b, 3, RR, iddd, True, [0,1]))

0

0

0

0

0